# Precompte Calculated Fields
Tableau be slow af so use zippy Python to precompute everything and store it. Yeah, Tableau is that bad that I called _Python_ fast

## Load the raw, scraped data

In [ ]:
import json
with open('scrapped.json', 'r') as fh:
    data = json.load(fh)
data[0]

### Create test user to validate upon

In [ ]:
user = lambda: data[0].copy()

## Basic Cleaning
This will all be outsourced to another script down the line.

### Fix year / dates
People sometimes don't include the month in their profiles

In [ ]:
def year(info):
    for job in info['professional']:
        if not job['start_year']:
            job['start_year'] = int(job['start_month'])
            job['start_month'] = 'Jan'
        else:
            job['start_year'] = int(job['start_year'])
            
        if not job['end_year']:
            job['end_year'] = int(job['end_month'])
            job['end_month'] = 'Jan'
        else:
            job['end_year'] = int(job['end_year'])
    return info

data = [year(d) for d in data]
data[0]

## Create Derived Features
I'll first create a bunch of helper functions + basic test--and then I'll assign everything at the very end.

### Years Since First Job
Show how long the person spent in each job

In [ ]:
def years_since_first_job(info):
    first_job = info['professional'][0]['start_year']
    
    for job in info['professional']:
        # HACK NEED TO FIX
        job['years_since_start'] = job['start_year'] - first_job
    
    return info

Validate

In [ ]:
years_since_first_job(user())

### Job Tenure
How long the person has been at their current job.

In [ ]:
def job_tenure(info):
    for job in info['professional']:
        # HACK NEED TO FIX
        job['tenure'] = max(job['end_year'] - job['start_year'], 1)
    return info

Validate

In [ ]:
job_tenure(user())

## TODO: Derived Features
Just some cool ideas that I can't implement now, but might explore down the line.

### Company Tenure / Satisfaction
Show how long the person spent at each company.

### Years Since WPI
Basically the same thing as `Years Since First Job`, but this is assuming that the data is theta-joined with registrar data to get immediate post-WPI insights.

Interesting insights could also be `Years since WPI`, and `Years since Last Degree`

## Merge all modifications

Create a super simple, parallelizable function

In [ ]:
def post(d):
    d = years_since_first_job(d)
    d = job_tenure(d)
    return d

Since the size is small, we can just process this bad boy sequentially. TODO: parallelize when dealing with the full data set.

In [ ]:
new = []
for d in data:
    try:
        new.append(post(d.copy()))
    except:
        print(d)
        raise
new[0]

Write to json

In [ ]:
with open('processed.json', 'w') as fh:
    json.dump(new, fh)